In [1]:
import openpathsampling as paths
import numpy as np

Load first frame from test pdb.

In [2]:
template = paths.tools.snapshot_from_pdb('../data/Alanine_solvated.pdb')

Create a complicated function.

In [3]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [4]:
cv = paths.CV_Function('func1', dist, center=center, np=np)

Create storage to test save and load.

In [5]:
storage = paths.storage.Storage('can_be_deleted.nc', mode='w', template=template)

Save CV

In [6]:
storage.save(cv);

Test function for reversed template.

In [7]:
print dist(template.reversed, center, np)

-1.08320000023


Check if function is time reversible.

In [8]:
assert dist(template, center, np) == dist(template.reversed, center, np)

Get CV parameters for the function.

In [9]:
print paths.collectivevariable.CV_Callable.parameters_from_template(dist, template, center=center, np=np)

{'c': <function dist at 0x10f24bcf8>, 'cv_return_simtk_unit': None, 'cv_time_reversible': True, 'cv_requires_lists': False, 'cv_return_shape': None, 'cv_return_type': 'numpy.float64'}


Create another CV. This time using the `from_template` function.

In [10]:
cv2 = paths.CV_Function.from_template('func2', dist, template, center=center, np=np)

Try both CVs. And note that the first CV is a list of numpy scalars of numpy.float64 while the second is numpy.array with two values.

In [11]:
print cv([template, template])

[-1.0832000002264977, -1.0832000002264977]


In [12]:
print type(cv([template, template]))
print type(cv([template, template])[0])

<type 'list'>
<type 'numpy.float64'>


In [13]:
print cv2([template, template])

[-1.0832 -1.0832]


In [14]:
print type(cv2([template, template]))
print type(cv2([template, template])[0])

<type 'numpy.ndarray'>
<type 'numpy.float64'>


Save also the second CV and sync all cached values.

In [15]:
storage.save(cv2)
cv2.sync()

In [16]:
#! skip
print storage.cvs.variables['json'][0]

{"_cls": "CV_Function", "_dict": {"simtk_unit": null, "store_cache": true, "name": "func1", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTMtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB", "_module_vars": [], "_global_vars": []}, "return_shape": null, "kwargs": {"np": {"_import": "numpy"}, "center": 1}, "requires_lists": false, "return_type": "float", "time_reversible": false}}


In [17]:
cv3 = storage.cvs.vars['json'][0]
print cv3.return_type

float


In [18]:
res = cv3([template, template, template])
assert(res[0] == res[1] == res[2])

In [19]:
cv4 = storage.cvs.vars['json'][1]
print cv4.return_type

numpy.float64


In [20]:
print storage.variables['snapshots_cv_0_values']

<type 'netCDF4._netCDF4.Variable'>
float32 snapshots_cv_0_values(snapshots)
    var_type: float
    maskable: True
unlimited dimensions: snapshots
current shape = (2,)
filling on, default _FillValue of 9.96920996839e+36 used



In [21]:
res = cv4([template, template, template])
assert(res[0] == res[1] == res[2])

In [22]:
print storage.variables['snapshots_cv_1_values']

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_1_values(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (2,)
filling on, default _FillValue of 9.96920996839e+36 used



In [23]:
print storage.vars['snapshots_cv_0_values'][:]

[None, None]


Sync again.

In [24]:
cv.sync()
cv2.sync()

In [25]:
print storage.vars['snapshots_cv_0_values'][:]

[-1.0831999778747559, None]


The first CV stores values as python floats which is the default. Since our function returns a numpy.float64 scalar we get floats back, once we load a cv and load the cache from disk.

In [26]:
print type(storage.vars['snapshots_cv_0_values'][0])

<type 'float'>


Note that the second CV, which had `time_reversible=True` will automatically compute the values for the reversed snapshot, too. So after syncing we see two equal values.

In [27]:
print storage.vars['snapshots_cv_1_values'][:]

[-1.0832000002264977, -1.0832000002264977]


And this is also numpy.float64 as was the original.

In [28]:
print type(storage.vars['snapshots_cv_1_values'][0])

<type 'numpy.float64'>


Last, check that really CV has one cached value, while CV2 has two.

In [29]:
print cv._store_dict.cache

LRUCache(1/0 of 100000/0)


In [30]:
print cv2._store_dict.cache

LRUCache(2/0 of 100000/0)


In [31]:
storage.cvs.cache.clear()